In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('bikers.db') # Create a database and connect to it
cursor = conn.cursor()

In [ ]:
df = pd.read_csv('data/BikerDatav2.csv', parse_dates=['start_time'])
df.info()

TypeError: read_csv() got an unexpected keyword argument 'parse_date'

In [ ]:
table_name = 'bikers'
int_cols = ['trip_id', 'start_station_id', 'end_station_id', 'duration_minutes']
timestamp_cols = ['start_time']

columns_with_types = ", ".join([
    f"{col.replace(' ', '_')} " +
    ("INTEGER" if col in int_cols else
     "TIMESTAMP" if col in timestamp_cols else
     "STRING")
    for col in df.columns
])

create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_with_types});"

cursor.execute(create_table_query)
cursor.fetchall()

[]

In [ ]:
cursor.execute('pragma table_info(bikers);')
cursor.fetchall()

[(0, 'trip_id', 'INTEGER', 0, None, 0),
 (1, 'subscriber_type', 'STRING', 0, None, 0),
 (2, 'bikeid', 'STRING', 0, None, 0),
 (3, 'start_time', 'TIMESTAMP', 0, None, 0),
 (4, 'start_station_id', 'INTEGER', 0, None, 0),
 (5, 'start_station_name', 'STRING', 0, None, 0),
 (6, 'end_station_id', 'INTEGER', 0, None, 0),
 (7, 'end_station_name', 'STRING', 0, None, 0),
 (8, 'duration_minutes', 'INTEGER', 0, None, 0)]

In [ ]:
for index, row in df.iterrows():
    values = ", ".join([f'"{row_item}"' for row_item in row])
    insert_sql = f"INSERT INTO {table_name} ({', '.join(df.columns.str.replace(' ', '_'))}) VALUES ({values})"
    cursor.execute(insert_sql)


In [ ]:
cursor.execute('SELECT COUNT(*) FROM bikers')
cursor.fetchall()

[(5619,)]

In [ ]:
df.shape

(5619, 9)

In [ ]:
conn.commit()

- On which day of the week do we on average have the longest trip?
-  What month/year has the most bike trips and what is the count of the trips?
- In the same table, return which particular trip has longest duration and the trip that has
    the shortest duration (return all the information(columns) on the table for this record)
    If more than 1 record has the same duration, return the earliest trip [start time]
    NB: Exclude 'Missing' and 'Stolen' as values in the end_station_name column.
    Exclude trips that start and end at the same station.
    Your final output will be 2 rows

In [ ]:
# Execute SQL query
query1 = """
SELECT 
    strftime('%w', substr(start_time, 1, 19)) AS weekday,
    strftime('%A', substr(start_time, 1, 19)) AS weekday_name,
    AVG(duration_minutes) AS avg_duration
FROM bikers
GROUP BY weekday, weekday_name
ORDER BY avg_duration DESC
LIMIT 1;
"""
cursor.execute(query1)
cursor.fetchall()

[('0', None, 79.10338517840805)]

In [ ]:
query2 = """
SELECT 
    strftime('%Y', start_time) AS year,
    strftime('%m', start_time) AS month,
    COUNT(*) AS trip_count
FROM bikers
WHERE end_station_name NOT IN ('Missing', 'Stolen')
  AND start_station_id != end_station_id
GROUP BY year, month
ORDER BY trip_count DESC
LIMIT 1;
"""

cursor.execute(query2)
cursor.fetchall()

[(None, None, 2676)]

In [ ]:
query3 = """
WITH cleaned_data AS (
    SELECT *
    FROM bikers
    WHERE end_station_name NOT IN ('Missing', 'Stolen')
      AND start_station_id != end_station_id
),
ranked_trips AS (
    SELECT *,
        RANK() OVER (ORDER BY duration_minutes ASC, start_time ASC) AS shortest_rank,
        RANK() OVER (ORDER BY duration_minutes DESC, start_time ASC) AS longest_rank
    FROM cleaned_data
)
SELECT * FROM ranked_trips
WHERE shortest_rank = 1
   OR longest_rank = 1;
"""

cursor.execute(query3)
cursor.fetchall()

[(21473408,
  'Pay-as-you-ride',
  460,
  '2020-01-15 09:14:08 UTC',
  2822,
  'East 6th/Robert T. Martinez',
  2544,
  'East 6th/Pedernales',
  2,
  1,
  2462),
 (21577822,
  '24 Hour Walk Up Pass',
  2095,
  '2020-02-16 04:37:00 UTC',
  4054,
  'Rosewood/Chicon',
  4058,
  'Hollow Creek/Barton Hills',
  11810,
  2676,
  1)]

In [ ]:
query4="""
SELECT start_time, strftime('%A', start_time)
FROM bikers
LIMIT 5;
"""
cursor.execute(query4)
cursor.fetchall()

[('2020-12-10 14:52:24 UTC', None),
 ('2020-12-18 15:43:07 UTC', None),
 ('2020-12-20 15:44:21 UTC', None),
 ('2021-08-08 17:31:07 UTC', None),
 ('2021-08-03 14:34:41 UTC', None)]